In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras import backend as K
from keras.models import load_model
import keras.utils
import numpy as np
import pickle

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16 = VGG16(weights='imagenet', include_top=True)
#model_vgg16_conv.summary()

# Load ImageNet validation data and split it into Xpr and Xtest

# Generate dummy data as Xpr and Xtest
Xpr = np.random.random((300, 224, 224, 3))
Ypr = keras.utils.to_categorical(np.random.randint(10, size=(30000, 1)), num_classes=10)
Xtest = np.random.random((100, 224, 224, 3))
Ytest = keras.utils.to_categorical(np.random.randint(10, size=(10000, 1)), num_classes=10)


# GET THE OUTPUT OF EACH LAYER AFTER TRAINING
inp = model_vgg16.input                                     # input placeholder
layIndx = np.array([10,22], dtype=np.dtype(np.int_))        # selected layer [2,3,9,10,20] and output layer [22]
modelSel = [model_vgg16.layers[i] for i in layIndx]
outputs = [layer.output for layer in modelSel]              # all selected layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
layer_outs = functor([Xpr, 1.])                             # compute on 2nd training set
layer_outs_test = functor([Xtest, 1.])                      # compute on test set

# SAVE THE OUTPUT OF LAYERS
with open('results/VGG_layer' + str(layIndx[0]) + '_outputs.dat','wb') as f:
    pickle.dump([layer_outs, layer_outs_test], f)
f.close()


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


In [2]:
layer_outs[0].shape

(300, 28, 28, 256)